> we need to clearly show the difference between ProGAN generator and the simple generator being used in DINet. make parallels to it.  

> For comparision sake we will limit the genertor to 512x512 outputs instead of 1024x1024 used in ProGAN.


> we perform pixelwise normalization of the feature vectors after each Conv 3×3 layer in the generator.

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import torch
from model import Generator, Discriminator
from mmengine import Config

In [3]:
cfg = Config.fromfile("config.py")
cfg

Config (path: config.py): {'image_root': '/root/prakash/data/ffhq/thumbnails128x128/', 'start_train_at_img_size': 4, 'device': 'cuda', 'lr': 0.001, 'batch_sizes': [32, 32, 32, 16, 16, 16], 'image_size': 128, 'img_channels': 3, 'z_dim': 128, 'in_channels': 128, 'lambda_gp': 10, 'progressive_epochs': [30, 30, 30, 30, 30, 30], 'factors': [1, 1, 1, 1, 0.5, 0.25]}

In [4]:
x = Generator(cfg.z_dim, cfg.in_channels, cfg.factors)
x

Generator(
  (initial): Sequential(
    (0): PixelNorm()
    (1): ConvTranspose2d(128, 128, kernel_size=(4, 4), stride=(1, 1))
    (2): LeakyReLU(negative_slope=0.2)
    (3): WSConv2d(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (4): LeakyReLU(negative_slope=0.2)
    (5): PixelNorm()
  )
  (initial_rgb): WSConv2d(
    (conv): Conv2d(128, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (prog_blocks): ModuleList(
    (0-2): 3 x ConvBlock(
      (conv1): WSConv2d(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (conv2): WSConv2d(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (leaky): LeakyReLU(negative_slope=0.2)
      (pn): PixelNorm()
    )
    (3): ConvBlock(
      (conv1): WSConv2d(
        (conv): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (con

In [8]:
z = torch.randn((4, 128, 1, 1))
z.shape

torch.Size([4, 128, 1, 1])

In [10]:
for step, fac in enumerate(cfg.factors):
    print(2**(2+step), fac, x(z, 0, step).shape)

4 1 torch.Size([4, 3, 4, 4])
8 1 torch.Size([4, 3, 8, 8])
16 1 torch.Size([4, 3, 16, 16])
32 1 torch.Size([4, 3, 32, 32])
64 0.5 torch.Size([4, 3, 64, 64])
128 0.25 torch.Size([4, 3, 128, 128])


In [12]:
disc = Discriminator(cfg.in_channels, cfg.factors)
disc

Discriminator(
  (prog_blocks): ModuleList(
    (0): ConvBlock(
      (conv1): WSConv2d(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (conv2): WSConv2d(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (leaky): LeakyReLU(negative_slope=0.2)
      (pn): PixelNorm()
    )
    (1): ConvBlock(
      (conv1): WSConv2d(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (conv2): WSConv2d(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (leaky): LeakyReLU(negative_slope=0.2)
      (pn): PixelNorm()
    )
    (2-4): 3 x ConvBlock(
      (conv1): WSConv2d(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (conv2): WSConv2d(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding

In [13]:
z = torch.randn((4, 3, 128, 128))
z.shape

torch.Size([4, 3, 128, 128])

In [14]:
for step, fac in enumerate(cfg.factors):
    print(2**(2+step), fac, disc(z, 0, step).shape)

4 1 torch.Size([4, 15625])
8 1 torch.Size([4, 3721])
16 1 torch.Size([4, 841])
32 1 torch.Size([4, 169])
64 0.5 torch.Size([4, 25])
128 0.25 torch.Size([4, 1])


In [15]:
out = 0
for name, params in x.named_parameters():
    out+=params.numel()
out

1435634

In [16]:
out = 0
for name, params in disc.named_parameters():
    out+=params.numel()
out

1575937